# [Advent of Code 2022 Day ?]()

In [ ]:
from __future__ import annotations
import ipytest
import pytest
import sys
sys.path.append("..")
from ansi import *
from comp import *
ipytest.autoconfig()
PART_ONE_SENTINEL = 0x3f3f3f3f + 1
PART_TWO_SENTINEL = 0x3f3f3f3f + 2
run_doctest_for = lambda func: doctest.run_docstring_examples(func, globals())

## Test Cases

In [ ]:
PART_ONE_CASES: dict[str, dict[str, str | int]] = {
    "example": {
        "example1": 24,
    },
    "input": {
        "input1": 655,
    },
}
PART_ONE_INPUTS: dict[str, dict[str, str | int]] = {
    key: {} for key in PART_ONE_CASES.keys()
}
PART_ONE_OUTPUTS: dict[str, dict[str, str | int]] = {
    key: {} for key in PART_ONE_CASES.keys()
}

In [ ]:
PART_TWO_CASES: dict[str, dict[str, str | int]] = {
    "example": {
        "example1": 93,
    },
    "input": {
        "input1": PART_TWO_SENTINEL,  # Not 1013 or 8192, both are too low
    },
}
PART_TWO_INPUTS: dict[str, dict[str, str | int]] = {
    key: {} for key in PART_TWO_CASES.keys()
}
PART_TWO_OUTPUTS: dict[str, dict[str, str | int]] = {
    key: {} for key in PART_TWO_CASES.keys()
}

## Input Parsing

In [ ]:
class Model(BaseModel):
    data: Any

def parse_input_from_filename(filename: str) -> Context:
    lines = list(yield_line(filename))

    ctx = Context()
    ctx.input = []

    input_lines = ctx.input

    for idx, line in enumerate(lines):
        chunks = line.split(" -> ")
        grains = [Point(*intsep(thing, ",")) for thing in chunks]
        input_lines.append(grains)

    x_min = INF
    x_max = -INF
    y_min = INF
    y_max = -INF

    for point in chain.from_iterable(input_lines):
        x, y = point
        x_min = min(x_min, x)
        x_max = max(x_max, x)
        y_min = min(y_min, y)
        y_max = max(y_max, y)

    print(f"{x_min=} {x_max=} {y_min=} {y_max=}")

    ctx.x_min = x_min
    ctx.x_max = x_max
    ctx.y_min = y_min
    ctx.y_max = y_max

    return ctx

In [ ]:
%%ipytest -xrPvvvvv
@pytest.mark.parametrize("test_file_name", PART_ONE_CASES["example"].keys() | PART_TWO_CASES["example"].keys())
def test_parsing_examples(test_file_name):
    for entity in parse_input_from_filename(test_file_name).input:
        enable_logging()
        log(f"{entity}")

In [ ]:
%%ipytest -xrPvvvvv
@pytest.mark.parametrize("test_file_name", PART_ONE_CASES["input"].keys() | PART_TWO_CASES["input"].keys())
def test_parsing_inputs(test_file_name):
    for entity in parse_input_from_filename(test_file_name).input:
        enable_logging()
        log(f"{entity}")

## Helper Functions

### Helper 1

In [ ]:
%%ipytest -xrPvvvvv

def get_line_coords(coords: list[Point]) -> set[Point]:
    assert len(coords) > 1, f"coords should not be empty"

    points = set(coords)

    for i in range(len(coords) - 1):
        x1, y1 = coords[i]
        x2, y2 = coords[i + 1]
        if x1 == x2:
            if y1 > y2:
                while y1 > y2:
                    y2 += 1
                    points.add(Point(x1, y2))
            elif y1 < y2:
                while y1 < y2:
                    y1 += 1
                    points.add(Point(x1, y1))
            else:
                raise ValueError(f"These two points are the same!")
        elif y1 == y2:
            if x1 > x2:
                while x1 > x2:
                    x2 += 1
                    points.add(Point(x2, y1))
            elif x1 < x2:
                while x1 < x2:
                    x1 += 1
                    points.add(Point(x1, y1))
            else:
                raise ValueError(f"These two points are the same!")
        else:
            raise ValueError(f"Not possible to draw a non-diagonal line from {x1=} {y1=} -> {x2=} {y2=}")

    return points

def test_helper_1() -> None:
    assert get_line_coords([Point(498, 4), Point(498, 6), Point(496, 6)]) == {Point(498, 6), Point(498, 5), Point(498, 4), Point(497, 6), Point(496, 6)}
    assert len(get_line_coords([Point(503, 4), Point(502, 4), Point(502, 9), Point(494, 9)])) == 15

### Helper 2

In [ ]:
%%ipytest -xrPvvvvv

def draw_board(board: list[list[str]]) -> None:
    for i in range(len(board)):
        print(f"{i: 2} {board[i]}")

def test_helper_2() -> None:
    draw_board(
        [
            list("......+..."),
            list(".........."),
            list(".........."),
            list(".........."),
            list("....#...##"),
            list("....#...#."),
            list("..###...#."),
            list("........#."),
            list("........#."),
            list("#########."),
        ]
    )

### Helper 3

In [ ]:
%%ipytest -xrPvvvvv

def board_to_string(board: list[list[str]]) -> str:
    return "\n".join("".join(row) for row in board)

def drop_grain(_board: list[list[str]], point: Point) -> list[list[str]]:
    board = copy.deepcopy(_board)
    x, y = point
    # assert board[x][y] == "+", f"Starting point {point} not marked with plus sign..."
    # assert board[x + 1][y] == ".", f"Can't spawn new grain of sand below spawn point:\n{board_to_string(board)}"
    # if board[x + 1][y] != ".":
    #     return board

    grain = Point(x, y)
    log(f"Spawned grain at {grain}")
    iterations = 0
    while True:
        iterations += 1
        if iterations >= 1000000:
            raise RuntimeError(f"Too many iterations - infinite loop detected")
        next_grain = grain + (1, 0)
        nx, ny = next_grain
        try:
            if board[nx][ny] == ".":
                log(f"Grain just passed through the air at {next_grain}")
                grain = next_grain
            elif board[nx][ny] in {"#", "o"}:
                log(f"{grain} will attempt to go left")
                left_grain = grain + (1, -1)
                nx, ny = left_grain
                if board[nx][ny] == ".":
                    grain = left_grain
                    continue

                log(f"{grain} will attempt to go right")
                right_grain = grain + (1, 1)
                nx, ny = right_grain
                if board[nx][ny] == ".":
                    grain = right_grain
                    continue

                log(f"Grain will stop at {grain} as it has hit solid ground")
                x, y = grain
                board[x][y] = "o"
                return board
            else:
                raise RuntimeError(f"weird...")
        except IndexError:
            log(f"Out of range! Seems every grain from this point on is getting swept away")
            return board


def test_helper_3() -> None:
    disable_logging()
    result = drop_grain(
        [
            list("......+..."),
            list(".........."),
            list(".........."),
            list(".........."),
            list("....#...##"),
            list("....#...#."),
            list("..###...#."),
            list("........#."),
            list("........#."),
            list("#########."),
        ],
        Point(0, 6),
    )
    expected = [
        list("......+..."),
        list(".........."),
        list(".........."),
        list(".........."),
        list("....#...##"),
        list("....#...#."),
        list("..###...#."),
        list("........#."),
        list("......o.#."),
        list("#########."),
    ]
    assert result == expected, f"Expected:\n{board_to_string(expected)}\nActual:\n{board_to_string(result)}"
    for _ in range(23):
        new_result = drop_grain(result, Point(0, 6))
        result = new_result
    assert result == (new_res := [
        list("......+..."),
        list(".........."),
        list("......o..."),
        list(".....ooo.."),
        list("....#ooo##"),
        list("...o#ooo#."),
        list("..###ooo#."),
        list("....oooo#."),
        list(".o.ooooo#."),
        list("#########."),
    ]), f"Expected:\n{board_to_string(new_res)}\nActual:\n{board_to_string(result)}"

### Helper 4

In [ ]:
%%ipytest -xrPvvvvv

def saturate(grid: list[list[str]], start: Point) -> list[list[str]]:
    curr_board = grid
    iterations = 0
    while True:
        if iterations >= 10000000:
            raise Exception(f"Too many iterations")
        iterations += 1
        next_board = drop_grain(curr_board, start)
        if next_board == curr_board:
            return next_board
        curr_board = next_board
    return next_board

def test_saturate() -> None:
    result = saturate(
        [
            list("......+..."),
            list(".........."),
            list(".........."),
            list(".........."),
            list("....#...##"),
            list("....#...#."),
            list("..###...#."),
            list("........#."),
            list("........#."),
            list("#########."),
        ],
        Point(0, 6),
    )
    print(board_to_string(result))

### Helper 5

In [ ]:
%%ipytest -xrPvvvvv

def count_grains(grid: list[list[str]]) -> int:
    count = 0
    for i in range(len(grid)):
        for j in range(len(grid[i])):
            if grid[i][j] == "o":
                count += 1
    return count

def test_count_grains() -> None:
    assert count_grains(saturate(
        [
            list("......+..."),
            list(".........."),
            list(".........."),
            list(".........."),
            list("....#...##"),
            list("....#...#."),
            list("..###...#."),
            list("........#."),
            list("........#."),
            list("#########."),
        ],
        Point(0, 6),
    )) == 24

### Helper 6

In [ ]:
%%ipytest -xrPvvvvv
# x_min=494 x_max=503 y_min=4 y_max=9

def initialize_board(points: list[Point], x_min: int, x_max: int, y_min: int, y_max: int) -> list[list[str]]:
    x_diff = x_max - x_min + 1
    y_diff = y_max - y_min + 1
    res = []

    height_offset = 4

    for i in range(y_diff + height_offset):
        res.append(["."] * (x_diff))

    for x, y in points:
        res[y - y_min + height_offset][x - x_min] = "#"

    return res

def test_initialize_board() -> None:
    points = get_line_coords([Point(498, 4), Point(498, 6), Point(496, 6)]) | get_line_coords([Point(503, 4), Point(502, 4), Point(502, 9), Point(494, 9)])

## Main Function

In [ ]:
def solve(part: int, filename: str) -> int:
    ctx = parse_input_from_filename(filename)
    input = ctx.input
    if part == 1:
        return 0
        x_min, x_max, y_min, y_max = ctx.x_min, ctx.x_max, ctx.y_min, ctx.y_max

        all_rocks = set()
        for points in input:
            all_rocks |= get_line_coords(points)

        board = initialize_board(all_rocks, x_min, x_max, y_min, y_max)
        # print(len(board), len(board[0]))
        board[0][500 - x_min] = "+"

        board = saturate(board, Point(0, 500 - x_min))
        # print(board_to_string(board))

        return count_grains(board)
    if part == 2:
        x_min, x_max, y_min, y_max = ctx.x_min, ctx.x_max, ctx.y_min, ctx.y_max

        offset = 160  # 80 took 3 minutes, 160 (22683) took 13, no 22684, 640 took 45
        input.append([Point(x_min - offset, y_max + 2), Point(x_max + offset, y_max + 2)])

        x_min = x_min - offset
        x_max = x_max + offset
        y_max += 2

        all_rocks = set()
        for points in input:
            all_rocks |= get_line_coords(points)

        board = initialize_board(all_rocks, x_min, x_max, y_min, y_max)
        # print(len(board), len(board[0]))
        # board[0][500 - x_min] = "+"
        # print(board_to_string(board))

        board = saturate(board, Point(0, 500 - x_min))
        # print(board_to_string(board))

        return count_grains(board)
    else:
        raise Exception(f"Invalid part: {part}")

### Part 1

In [ ]:
%%ipytest -xrPvvvvv
@pytest.mark.parametrize("test_file_name, test_expected_output", PART_ONE_CASES["example"].items())
def test_part_one_examples(test_file_name, test_expected_output):
    test_actual_output = solve(1, test_file_name)
    PART_ONE_OUTPUTS["example"][test_file_name] = test_actual_output
    failure_message = "Did you forget to calibrate the example test case?" if (
        test_expected_output == PART_ONE_SENTINEL
    ) else f"Failed example test case: expected {test_expected_output} but got {test_actual_output}"
    assert test_actual_output == test_expected_output, failure_message

@pytest.mark.parametrize("test_file_name, test_expected_output", PART_ONE_CASES["input"].items())
def test_part_one_inputs(test_file_name, test_expected_output):
    test_actual_output = solve(1, test_file_name)
    PART_ONE_OUTPUTS["input"][test_file_name] = test_actual_output
    failure_message = f"Candidate answer {test_actual_output} found" if (
        test_expected_output == PART_ONE_SENTINEL
    ) else f"Failed input test case: expected {test_expected_output} but got {test_actual_output}"
    assert test_actual_output == test_expected_output, failure_message

### Part 2

In [ ]:
%%ipytest -xrPvvvvv
@pytest.mark.parametrize("test_file_name, test_expected_output", PART_TWO_CASES["example"].items())
def test_part_two_examples(test_file_name, test_expected_output):
    test_actual_output = solve(2, test_file_name)
    PART_TWO_OUTPUTS["example"][test_file_name] = test_actual_output
    failure_message = "Did you forget to calibrate the example test case?" if (
        test_expected_output == PART_TWO_SENTINEL
    ) else f"Failed example test case: expected {test_expected_output} but got {test_actual_output}"
    assert test_actual_output == test_expected_output, failure_message

@pytest.mark.parametrize("test_file_name, test_expected_output", PART_TWO_CASES["input"].items())
def test_part_two_inputs(test_file_name, test_expected_output):
    test_actual_output = solve(2, test_file_name)
    PART_TWO_OUTPUTS["input"][test_file_name] = test_actual_output
    failure_message = f"Candidate answer {test_actual_output} found" if (
        test_expected_output == PART_TWO_SENTINEL
    ) else f"Failed input test case: expected {test_expected_output} but got {test_actual_output}"
    assert test_actual_output == test_expected_output, failure_message

dadada